In [0]:
import csv
import pandas as pd
import numpy as np
from collections import Counter
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from numpy import array

In [3]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM

Using TensorFlow backend.


In [0]:
import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df


In [4]:
from google.colab import files

uploaded = files.upload()

Saving text.csv to text.csv


In [4]:
text=pd.read_csv('text.csv')
text.head()

,message,response
0,I'd like to book a trip to Atlantis from Capri...,"Hi...I checked a few options for you, and unfo..."
1,"Yes, how about going to Neverland from Caprica...",I checked the availability for this date and t...
2,I have no flexibility for dates... but I can l...,I checked the availability for that date and t...
3,"Hello, I am looking to book a vacation from Go...","Hi. Sorry, I can't find any trips from Gotham ..."
4,What about a trip from Gotham City to Neverlan...,"Sorry, I cannot find any trips leaving from Go..."


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stop_words = set(stopwords.words("english"))

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

from tqdm import tqdm
preprocessed_que = []
preprocessed_ans = []
# tqdm is for printing the status bar
for sentance in tqdm(text['message'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    preprocessed_que.append(sentance.strip())

for sentance in tqdm(text['response'].values):
    sentance = re.sub(r"http\S+", "", str(sentance))
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    preprocessed_ans.append(sentance.strip())

100%|██████████| 9579/9579 [00:02<00:00, 4295.98it/s]


In [7]:
text['message'] = preprocessed_que
text['response'] = preprocessed_ans
text.head()

,message,response
0,I would like to book a trip to Atlantis from C...,Hi I checked a few options for you and unfortu...
1,Yes how about going to Neverland from Caprica ...,I checked the availability for this date and t...
2,I have no flexibility for dates but I can leav...,I checked the availability for that date and t...
3,Hello I am looking to book a vacation from Got...,Hi Sorry I can not find any trips from Gotham ...
4,What about a trip from Gotham City to Neverlan...,Sorry I cannot find any trips leaving from Got...


In [0]:
train=text[0:9000]
test=text[9000: ]

In [0]:
train_input=train['message'].values
test_input=test['message'].values

train_target=train['response'].values
test_target=test['response'].values

In [0]:
max_input_len = 60

embed_size = 100

tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(list(train_input))

train_q = tokenizer.texts_to_sequences(train_input)
test_q = tokenizer.texts_to_sequences(test_input)

x_train = pad_sequences(train_q, maxlen=max_input_len)
x_test = pad_sequences(test_q, maxlen=max_input_len)


In [0]:
max_output_len=60
t = Tokenizer(num_words=500)
t.fit_on_texts(list(train_target))

train_a = t.texts_to_sequences(train_target)
test_a = t.texts_to_sequences(test_target)

y_train = pad_sequences(train_a,maxlen=max_output_len)
y_test = pad_sequences(test_a,maxlen=max_output_len)

    
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [0]:
X1, X2, y = list(), list(), list()

target_in = y_train
    
src_encoded = to_categorical(x_train)
tar_encoded = to_categorical(y_train)
tar2_encoded = to_categorical(target_in)

X1.append(src_encoded)
X2.append(tar2_encoded)
y.append(tar_encoded)


In [30]:
X1=array(X1).reshape(9000,60,500)
X2=array(X2).reshape(9000,60,500)
y=array(y).reshape(9000,60,500)

print(X1.shape, X2.shape, y.shape)
print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

(9000, 60, 500) (9000, 60, 500) (9000, 60, 500)
X1=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 19, 29, 2, 26, 5, 41, 2, 11, 27, 438, 44, 8, 74, 1, 15, 5, 496, 51, 18], X2=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 180, 7, 2, 268, 142, 8, 1, 5, 99, 32, 23, 50, 9, 62, 105, 31, 20, 11, 1, 26, 4, 51, 75, 431, 93, 165], y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 180, 7, 2, 268, 142, 8, 1, 5, 99, 32, 23, 50, 9, 62, 105, 31, 20, 11, 1, 26, 4, 51, 75, 431, 93, 165]


In [31]:
def define_models(n_input, n_output, n_units):
# define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
# define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
# define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
# return all models
    return model, encoder_model, decoder_model
 
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
    # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
    # store prediction
        output.append(yhat[0,0,:])
    # update state
        state = [h, c]
    # update target sequence
        target_seq = yhat
    return array(output)
 
# configure problem
n_features = 500
n_steps_in = 60
n_steps_out = 60
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, None, 500)    0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, None, 500)    0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 128), (None, 322048      input_9[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, None, 128),  322048      input_10[0][0]                   
                                                                 lstm_5[0][1]                     
          

In [32]:
history=train.fit([X1, X2], y, epochs=200,validation_split=0.2)

Train on 7200 samples, validate on 1800 samples
Epoch 1/200
7200/7200 [==============================] - 39s 5ms/step - loss: 1.7670 - acc: 0.7725 - val_loss: 1.2908 - val_acc: 0.7803
Epoch 2/200
7200/7200 [==============================] - 38s 5ms/step - loss: 1.1303 - acc: 0.7989 - val_loss: 1.0007 - val_acc: 0.8315
Epoch 3/200
7200/7200 [==============================] - 38s 5ms/step - loss: 0.8228 - acc: 0.8707 - val_loss: 0.6698 - val_acc: 0.9089
Epoch 4/200
7200/7200 [==============================] - 37s 5ms/step - loss: 0.5209 - acc: 0.9273 - val_loss: 0.4304 - val_acc: 0.9398
Epoch 5/200
7200/7200 [==============================] - 38s 5ms/step - loss: 0.3395 - acc: 0.9539 - val_loss: 0.3012 - val_acc: 0.9598
Epoch 6/200
7200/7200 [==============================] - 37s 5ms/step - loss: 0.2368 - acc: 0.9699 - val_loss: 0.2210 - val_acc: 0.9716
Epoch 7/200
7200/7200 [==============================] - 37s 5ms/step - loss: 0.1715 - acc: 0.9782 - val_loss: 0.1673 - val_acc: 0.9775


In [33]:
train.save("lstm_model.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_6 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_5/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'lstm_5/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [0]:
train.save_weights("lstm_weights.h5")

In [0]:
X_1, X_2, Y = list(), list(), list()

target_in = y_test
    
src_encoded = to_categorical(x_test,num_classes=500)
tar_encoded = to_categorical(y_test)
tar2_encoded = to_categorical(target_in)

X_1.append(src_encoded)
X_2.append(tar2_encoded)
Y.append(tar_encoded)

X_1=array(X_1).reshape(579,60,500)
X_2=array(X_2).reshape(579,60,500)
Y=array(Y).reshape(579,60,500)


In [0]:
x1=X_1[0:1:,]
x2=X_1[1:2:,]
x3=X_1[2:3:,]
x4=X_1[3:4:,]

In [43]:
target = predict_sequence(infenc, infdec, x2, n_steps_out, n_features)
print('X=%s y=%s, yhat=%s' % (one_hot_decode(X_1[1]), one_hot_decode(Y[1]),one_hot_decode(target)))
y_pred=one_hot_decode(target)
a=t.word_index
b=list(a.keys())
reply=[]
for i in range(2115):
    for j in range(60):
        if y_pred[j]==i+1:
            reply.append(b[i])

print("*****************EMAIL_BODY********************\n")

print(test_input[1])

print("*******************Suggested Reply**************")

print(' '.join(reply))

print("*******************Actual Reply******************")
print(test_target[1])

X=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 25, 55, 87, 151, 155, 14, 428] y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 3, 57, 2, 14, 38, 6, 40, 127, 21, 61, 5, 71, 21, 44, 3, 19, 87, 16, 34, 21, 30, 54, 8, 78, 16, 1], yhat=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
*****************EMAIL_BODY********************

Is there anything better than stars in Cordoba
*******************Suggested Reply**************

*******************Actual Reply******************
In Cordoba the Everland has a star rating is available within your dates and also sits your budget The package price of will cover your day stay for all of you
